In [ ]:
!pip3 install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00


In [ ]:
# Importing necessary libraries and modules

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login


In [ ]:
import os

# Get the Hugging Face token from Colab secrets
hf_token = os.environ.get('HF_TOKEN')

from huggingface_hub import login

# Log in to the Hugging Face Hub
login(token=hf_token)

# Loading the dataset
dataset = load_dataset("likaixin/InstructCoder")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/108391 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5708 [00:00<?, ? examples/s]

In [ ]:
# Configuring the data type for computations
compute_dtype = getattr(torch, "float16")

# Configuring Bits and Bytes for model quantization
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
# Setting the model name
model_name='microsoft/phi-2'

# Configuring the device map
device_map = {"": 0}

# Loading the pre-trained model with specific configurations
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Loading the tokenizer with specific configurations
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,
                                          padding_side="left",
                                          add_eos_token=True,add_bos_token=True,use_fast=False)

# Setting the padding token to the EOS token
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Setting the device for computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Defining the text generation function
def gen(model, prompt, max_new_tokens=50):
    # Tokenizing the input prompt and moving the tensors to the specified device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generating the output from the model
    outputs = model.generate(inputs["input_ids"], max_new_tokens=max_new_tokens)

    # Decoding the generated tokens and returning the result, skipping special tokens
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
# Measuring the execution time
%%time


from transformers import set_seed

# Setting the random seed
seed = 42
set_seed(seed)

# Specifying the index of the example from the dataset
index = 100

# Extracting instruction, output, and input from the dataset
instruction = dataset['train'][index]['instruction']
output = dataset['train'][index]['output']
input = dataset['train'][index]['input']


# Formatting the prompt
formatted_prompt = f"Instruct: Based on given instruction and context, generate an appropriate response.\n{instruction}\nContext:\n{input}\nOutput:\n"


# Generating the response using the model
res = gen(original_model,formatted_prompt,index,)
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100)) # for formatting the dash line in print

# Printing the input prompt, baseline human output, and model-generated output
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN OUTPUT:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Based on given instruction and context, generate an appropriate response.
Improve efficiency of the `calculate_median` function by sorting the list first and then finding the median.
Context:
def calculate_median(ages):
    n = len(ages)
    if n % 2 == 0:
        return (ages[n//2] + ages[n//2 - 1]) / 2
    else:
        return ages[n//2]

ages = [25, 18, 21, 35, 40, 28, 22, 30]
print(calculate_median(ages))
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN OUTPUT:
def calculate_median(ages):
    sorted_ages = sorted(ages)
    n = len(sorted_ages)
    if n % 2 == 0:
        return (sorted_ages[n//2] + sorted_ages[n//2 - 1]) / 2
    else:
        return sorted_ages[n//2]

ages = [25, 18, 21, 35, 40, 28, 22, 30]
print(calculate_median(ages))

--------------------------------------------------

In [ ]:
# Function to format the sample with instruction and response
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Based on given instruction and context, generate an appropriate response: "
    INSTRUCTION_KEY = "### Context: "
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{sample['instruction']}" if 'instruction' in sample else None
    input_context = f"{sample['input']}" if 'input' in sample else None
    response = f"{RESPONSE_KEY}\n{sample['output']}" if 'output' in sample else None
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample


from functools import partial
from transformers import AutoTokenizer

# Function to get the maximum length supported by the model
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max length: {max_length}")
            break
    if not max_length:
        max_length = 4096
        print(f"Using default max length: {max_length}")
    return max_length

# Function to preprocess a batch by tokenizing it
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# Function to preprocess the dataset
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['instruction', 'output', 'input'],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset


## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

# Preprocess training and validation datasets
train_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['validation'])

Found max length: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/108391 [00:00<?, ? examples/s]

Map:   0%|          | 0/108391 [00:00<?, ? examples/s]

Filter:   0%|          | 0/108391 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/5708 [00:00<?, ? examples/s]

Map:   0%|          | 0/5708 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5708 [00:00<?, ? examples/s]

In [ ]:
# Function to print the number of trainable parameters in the model
def print_number_of_trainable_model_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Total parameters: {total_params:,}")
    return trainable_params

# Importing necessary libraries and modules
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Using the prepare_model_for_kbit_training method from PEFT
# Preparing the Model for QLoRA
original_model = prepare_model_for_kbit_training(original_model)

# Configuring LoRA (Low-Rank Adaptation)
config = LoraConfig(
    r=32, #Rank
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    task_type="CAUSAL_LM",
)
# Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

# Getting the PEFT model with the specified configuration
peft_model = get_peft_model(original_model, config)

# Print the number of trainable parameters
print_number_of_trainable_model_parameters(peft_model)

Trainable parameters: 20,971,520
Total parameters: 1,542,364,160


20971520

In [ ]:
# Setting the output directory for saving the model
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

# Importing the transformers library
import transformers

In [ ]:
# Setting training arguments for PEFT training
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    max_steps=400,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_dir="./logs",
    save_strategy="steps",
    logging_steps=25,
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=100,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
    push_to_hub=True,
)

# Disabling cache usage for the PEFT model
peft_model.config.use_cache = False

# Initializing the Trainer for PEFT model
peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Working out
peft_trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.116500,0.737540
200,0.704300,0.541103
300,0.650400,0.518693
400,0.644600,0.512481


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=400, training_loss=0.7269120192527772, metrics={'train_runtime': 2487.8171, 'train_samples_per_second': 5.145, 'train_steps_per_second': 0.161, 'total_flos': 5.40904573403136e+16, 'train_loss': 0.7269120192527772, 'epoch': 0.1180898959332792})

In [ ]:
# Importing necessary libraries and modules
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Setting the base model ID
base_model_id = "microsoft/phi-2"

# Loading the base model with specific configurations
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Loading the tokenizer for evaluation with specific configurations
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)

# Setting the padding token to the EOS token
eval_tokenizer.pad_token = eval_tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from peft import PeftModel

# Loading the PEFT model
peft_model = PeftModel.from_pretrained(base_model, "nglguarino/my-nlp-phi2-model")
# ft_model = PeftModel.from_pretrained(base_model,
    #"/content/peft-dialogue-summary-training-1719330073/checkpoint-925",torch_dtype=torch.float16,is_trainable=False)

adapter_config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

In [ ]:
%%time
from transformers import set_seed
set_seed(seed)

index = 200

# Extracting instruction, output, and input from the validation dataset
instruction = dataset['validation'][index]['instruction']
output = dataset['validation'][index]['output']
input = dataset['validation'][index]['input']

# Formatting the prompt
prompt = f"Instruct: Based on given instruction and context, generate an appropriate response.\n{instruction}\nContext:\n{input}\nOutput:\n"

# Generating the response using the PEFT model
peft_model_res = gen(peft_model,prompt,index,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
output_peft, success, result = peft_model_output.partition('###')

# Generating the response using the original model
res = gen(original_model,prompt,index,)
output_base = res[0].split('Output:\n')[1]
output_base, success, result = output_base.partition('###')

dash_line = '-'.join('' for x in range(100)) # dashed line

# Printing the input prompt, baseline human output, PEFT model output, and original model output
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE CODE:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{output_peft}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output_base}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Based on given instruction and context, generate an appropriate response.
Use type() instead of isinstance() for consistency.
Context:
def process_data(data):
    if isinstance(data, str):
        print("Processing string data...")
    elif isinstance(data, list):
        print("Processing list data...")
    elif isinstance(data, dict):
        print("Processing dictionary data...")
    else:
        print("Unsupported data type.")

data = "example"
process_data(data)
Output:

---------------------------------------------------------------------------------------------------
BASELINE CODE:
def process_data(data):
    if type(data) == str:
        print("Processing string data...")
    elif type(data) == list:
        print("Processing list data...")
    elif type(data) == dict:
        print("Processing dictionary data...")
    else:
        print("Unsupported dat

In [ ]:
# Installing necessary libraries
!pip install evaluate --q
!pip install rouge_score --q

In [ ]:
import evaluate

# Load BLEU and ROUGE metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
perplexity_metric = evaluate.load("perplexity")

# Outputs
knowledge_outputs = [output]  # Ground truth
base_model_outputs = [output_base]  # Base model output
fine_tuned_outputs = [output_peft]  # Fine-tuned model output

# Function to compute BLEU and ROUGE scores
def compute_metrics(references, hypotheses, model, tokenizer):
    # BLEU score
    bleu_score = bleu_metric.compute(predictions=hypotheses, references=[[ref] for ref in references])
    # ROUGE score
    rouge_score = rouge_metric.compute(predictions=hypotheses, references=references)

    # Prepare inputs for perplexity
    inputs = tokenizer(references, return_tensors='pt', padding=True, truncation=True)
    labels = inputs['input_ids']
    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
    perplexity = torch.exp(outputs.loss).item()

    return bleu_score, rouge_score, perplexity

# Compute metrics for base model
base_bleu_score, base_rouge_score, base_perplexity = compute_metrics(knowledge_outputs, base_model_outputs, original_model, tokenizer)

# Compute metrics for fine-tuned model
fine_tuned_bleu_score, fine_tuned_rouge_score, fine_tuned_perplexity = compute_metrics(knowledge_outputs, fine_tuned_outputs, peft_model, tokenizer)


# Print results
print("Base Model BLEU Score:", base_bleu_score)
print("Base Model ROUGE Score:", base_rouge_score)
print("Base Model Perplexity:", base_perplexity)
print("\nFine-tuned Model BLEU Score:", fine_tuned_bleu_score)
print("Fine-tuned Model ROUGE Score:", fine_tuned_rouge_score)
print("Fine-tuned Model Perplexity:", fine_tuned_perplexity)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Base Model BLEU Score: {'bleu': 0.01781985053485226, 'precisions': [0.6666666666666666, 0.55, 0.42105263157894735, 0.3333333333333333], 'brevity_penalty': 0.03741385136723659, 'length_ratio': 0.23333333333333334, 'translation_length': 21, 'reference_length': 90}
Base Model ROUGE Score: {'rouge1': 0.29166666666666663, 'rouge2': 0.2173913043478261, 'rougeL': 0.29166666666666663, 'rougeLsum': 0.29166666666666663}
Base Model Perplexity: 1.8174192905426025

Fine-tuned Model BLEU Score: {'bleu': 0.7662085158614671, 'precisions': [0.7692307692307693, 0.7672413793103449, 0.7652173913043478, 0.7631578947368421], 'brevity_penalty': 1.0, 'length_ratio': 1.3, 'translation_length': 117, 'reference_length': 90}
Fine-tuned Model ROUGE Score: {'rouge1': 0.8351648351648352, 'rouge2': 0.8314606741573033, 'rougeL': 0.8351648351648352, 'rougeLsum': 0.8351648351648352}
Fine-tuned Model Perplexity: 1.7543565034866333


In [ ]:
# Testing now for different indexes

index = 100
instruction = dataset['validation'][index]['instruction']
output = dataset['validation'][index]['output']
input = dataset['validation'][index]['input']

prompt = f"Instruct: Based on given instruction and context, generate an appropriate response.\n{instruction}\nContext:\n{input}\nOutput:\n"

peft_model_res = gen(peft_model,prompt,index,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
output_peft, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))

res = gen(original_model,prompt,index,)
output_base = res[0].split('Output:\n')[1]
output_base, success, result = output_base.partition('###')


print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE CODE:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{output_peft}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output_base}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Based on given instruction and context, generate an appropriate response.
Include a new optional parameter 'ignore_case' in the 'find_word' function that searches for a word in a string while ignoring case sensitivity.
Context:
def find_word(word, text):
    return word in text

text = "I love using Python for natural language processing."
word = "python"

print(find_word(word, text))
Output:

---------------------------------------------------------------------------------------------------
BASELINE CODE:
def find_word(word, text, ignore_case=False):
    if ignore_case:
        return word.lower() in text.lower()
    else:
        return word in text

text = "I love using Python for natural language processing."
word = "python"

print(find_word(word, text))
print(find_word(word, text, ignore_case=True))

-----------------------------------------------------------

In [ ]:
# Load BLEU and ROUGE metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
perplexity_metric = evaluate.load("perplexity")

# Outputs
knowledge_outputs = [output]  # Ground truth
base_model_outputs = [output_base]  # Base model output
fine_tuned_outputs = [output_peft]  # Fine-tuned model output

# Function to compute BLEU and ROUGE scores
def compute_metrics(references, hypotheses, model, tokenizer):
    # BLEU score
    bleu_score = bleu_metric.compute(predictions=hypotheses, references=[[ref] for ref in references])
    # ROUGE score
    rouge_score = rouge_metric.compute(predictions=hypotheses, references=references)

    # Prepare inputs for perplexity
    inputs = tokenizer(references, return_tensors='pt', padding=True, truncation=True)
    labels = inputs['input_ids']
    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
    perplexity = torch.exp(outputs.loss).item()

    return bleu_score, rouge_score, perplexity

# Compute metrics for base model
base_bleu_score, base_rouge_score, base_perplexity = compute_metrics(knowledge_outputs, base_model_outputs, original_model, tokenizer)

# Compute metrics for fine-tuned model
fine_tuned_bleu_score, fine_tuned_rouge_score, fine_tuned_perplexity = compute_metrics(knowledge_outputs, fine_tuned_outputs, peft_model, tokenizer)


# Print results
print("Base Model BLEU Score:", base_bleu_score)
print("Base Model ROUGE Score:", base_rouge_score)
print("Base Model Perplexity:", base_perplexity)
print("\nFine-tuned Model BLEU Score:", fine_tuned_bleu_score)
print("Fine-tuned Model ROUGE Score:", fine_tuned_rouge_score)
print("Fine-tuned Model Perplexity:", fine_tuned_perplexity)

Base Model BLEU Score: {'bleu': 0.689862807003448, 'precisions': [0.9855072463768116, 0.8823529411764706, 0.835820895522388, 0.7878787878787878], 'brevity_penalty': 0.7930380644189127, 'length_ratio': 0.8117647058823529, 'translation_length': 69, 'reference_length': 85}
Base Model ROUGE Score: {'rouge1': 0.9047619047619047, 'rouge2': 0.7804878048780488, 'rougeL': 0.8333333333333333, 'rougeLsum': 0.8809523809523809}
Base Model Perplexity: 2.144721508026123

Fine-tuned Model BLEU Score: {'bleu': 0.689862807003448, 'precisions': [0.9855072463768116, 0.8823529411764706, 0.835820895522388, 0.7878787878787878], 'brevity_penalty': 0.7930380644189127, 'length_ratio': 0.8117647058823529, 'translation_length': 69, 'reference_length': 85}
Fine-tuned Model ROUGE Score: {'rouge1': 0.9047619047619047, 'rouge2': 0.7804878048780488, 'rougeL': 0.8333333333333333, 'rougeLsum': 0.8809523809523809}
Fine-tuned Model Perplexity: 2.0568830966949463


In [ ]:
index = 500
instruction = dataset['validation'][index]['instruction']
output = dataset['validation'][index]['output']
input = dataset['validation'][index]['input']

prompt = f"Instruct: Based on given instruction and context, generate an appropriate response.\n{instruction}\nContext:\n{input}\nOutput:\n"

peft_model_res = gen(peft_model,prompt,index,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
output_peft, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))

res = gen(original_model,prompt,index,)
output_base = res[0].split('Output:\n')[1]
output_base, success, result = output_base.partition('###')


print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE CODE:\n{output}\n')
print(dash_line)
print(f'PEFT MODEL:\n{output_peft}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output_base}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Based on given instruction and context, generate an appropriate response.
Refactor the code to use 'assert' statements for checking preconditions and postconditions.
Context:
import random

def post_to_twitter(content):
    # Simulate posting to Twitter
    success = random.choice([True, False])
    if success:
        return "https://twitter.com/my_account/status/123456789"
    else:
        return None

def post_to_facebook(content):
    # Simulate posting to Facebook
    success = random.choice([True, False])
    if success:
        return "https://www.facebook.com/my_account/posts/123456789"
    else:
        return None

content = "Check out my new blog post!"
twitter_url = post_to_twitter(content)
facebook_url = post_to_facebook(content)

if twitter_url is not None:
    print(f"Posted to Twitter: {twitter_url}")
else:
    print("Failed to post to Twitter.")


In [ ]:
# Load BLEU and ROUGE metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
perplexity_metric = evaluate.load("perplexity")

# outputs
knowledge_outputs = [output]  # Ground truth
base_model_outputs = [output_base]  # Base model output
fine_tuned_outputs = [output_peft]  # Fine-tuned model output

# Function to compute BLEU and ROUGE scores
def compute_metrics(references, hypotheses, model, tokenizer):
    # BLEU score
    bleu_score = bleu_metric.compute(predictions=hypotheses, references=[[ref] for ref in references])
    # ROUGE score
    rouge_score = rouge_metric.compute(predictions=hypotheses, references=references)

    # Prepare inputs for perplexity
    inputs = tokenizer(references, return_tensors='pt', padding=True, truncation=True)
    labels = inputs['input_ids']
    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
    perplexity = torch.exp(outputs.loss).item()

    return bleu_score, rouge_score, perplexity

# Compute metrics for base model
base_bleu_score, base_rouge_score, base_perplexity = compute_metrics(knowledge_outputs, base_model_outputs, original_model, tokenizer)

# Compute metrics for fine-tuned model
fine_tuned_bleu_score, fine_tuned_rouge_score, fine_tuned_perplexity = compute_metrics(knowledge_outputs, fine_tuned_outputs, peft_model, tokenizer)


# Print results
print("Base Model BLEU Score:", base_bleu_score)
print("Base Model ROUGE Score:", base_rouge_score)
print("Base Model Perplexity:", base_perplexity)
print("\nFine-tuned Model BLEU Score:", fine_tuned_bleu_score)
print("Fine-tuned Model ROUGE Score:", fine_tuned_rouge_score)
print("Fine-tuned Model Perplexity:", fine_tuned_perplexity)

Base Model BLEU Score: {'bleu': 0.8216359728946345, 'precisions': [0.9395161290322581, 0.8704453441295547, 0.8130081300813008, 0.7551020408163265], 'brevity_penalty': 0.9760967695185692, 'length_ratio': 0.9763779527559056, 'translation_length': 248, 'reference_length': 254}
Base Model ROUGE Score: {'rouge1': 0.9309090909090909, 'rouge2': 0.8424908424908426, 'rougeL': 0.6836363636363636, 'rougeLsum': 0.9309090909090909}
Base Model Perplexity: 1.7625622749328613

Fine-tuned Model BLEU Score: {'bleu': 0.8214881547567825, 'precisions': [0.9471544715447154, 0.8775510204081632, 0.819672131147541, 0.7613168724279835], 'brevity_penalty': 0.9680027747782112, 'length_ratio': 0.968503937007874, 'translation_length': 246, 'reference_length': 254}
Fine-tuned Model ROUGE Score: {'rouge1': 0.9377289377289377, 'rouge2': 0.8487084870848708, 'rougeL': 0.6886446886446886, 'rougeLsum': 0.9377289377289377}
Fine-tuned Model Perplexity: 1.7204995155334473
